<a href="https://colab.research.google.com/github/grabuffo/BrainStim_ANN_fMRI_HCP/blob/main/notebooks/Simulate_PerGroup_DeltaFC_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulate Per-Group Models & Validate with Subject-Specific ΔFC

Generate synthetic TMS-fMRI sessions using target-specific models trained per-group.
For each subject, generate:
- Rest sessions: one per stim session (matched pairs)
- Stim sessions: using their target-specific model with real stimulation timing

Then validate by comparing empirical vs. simulated ΔFC = FC_stim - FC_rest per subject.

In [ ]:
# =========================
# 0) Mount Google Drive
# =========================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# =========================
# 1) Clone repo + imports
# =========================
import os, sys, pickle, json, math
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from collections import defaultdict

REPO_DIR = "/content/BrainStim_ANN_fMRI_HCP"
if not os.path.exists(REPO_DIR):
    !git clone https://github.com/grabuffo/BrainStim_ANN_fMRI_HCP.git
else:
    print("Repo already exists ✅")

sys.path.append(REPO_DIR)

from src.NPI import build_model, device
print("Torch device:", device)

In [ ]:
# =========================
# 2) Paths (EDIT IF NEEDED)
# =========================
BASE = "/content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data"

DATASET_EMP_PKL = os.path.join(BASE, "TMS_fMRI", "dataset_tian50_schaefer400_allruns.pkl")
GROUPING_SUMMARY_PKL = os.path.join(BASE, "TMS_fMRI", "target_regions_grouping_summary.pkl")

PREPROC_ROOT = os.path.join(BASE, "preprocessed_subjects_tms_fmri")
MODELS_DIR = os.path.join(PREPROC_ROOT, "trained_models_MLP_tms_fmri_pergroup")

OUT_DIR = os.path.join(PREPROC_ROOT, "ANN_vs_tms_fmri_pergroup")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PKL = os.path.join(OUT_DIR, "dataset_simulated_pergroup_ANN.pkl")

print("Empirical dataset:", DATASET_EMP_PKL, "| exists:", os.path.exists(DATASET_EMP_PKL))
print("Grouping summary:", GROUPING_SUMMARY_PKL, "| exists:", os.path.exists(GROUPING_SUMMARY_PKL))
print("Models directory:", MODELS_DIR, "| exists:", os.path.exists(MODELS_DIR))
print("Will save to:", OUT_PKL)

In [ ]:
# =========================
# 3) Load empirical dataset + grouping summary
# =========================
with open(DATASET_EMP_PKL, "rb") as f:
    dataset_emp = pickle.load(f)

with open(GROUPING_SUMMARY_PKL, "rb") as f:
    grouping_summary = pickle.load(f)

subjects_list = grouping_summary['subjects_list']
multi_subject_groups = grouping_summary['multi_subject_groups']
subject_targets_dict = grouping_summary['subject_targets']

print(f"Loaded {len(dataset_emp)} subjects")
print(f"Grouping: {len(multi_subject_groups)} multi-subject groups")

# Build mapping: subject_id -> target_id (from their stim sessions)
subject_to_target = {}
for target_id, group_info in multi_subject_groups.items():
    for sid in group_info['subject_ids']:
        subject_to_target[sid] = target_id

print(f"\nSubject to target mapping:")
for sid in range(len(subjects_list)):
    if sid in subject_to_target:
        print(f"  Subject {sid} ({subjects_list[sid]}): target {subject_to_target[sid]}")

In [ ]:
# =========================
# 4) Config
# =========================
S = 3
N = 450

TR_MODEL = 2.0          # model step in seconds
BURN_IN = 10           # steps (not saved)
NOISE_SIGMA = 0.3      # z-scored units
STIM_AMP = 1.0        # z-scored units

MAP_MODE = "round"      # "round" | "floor" | "ceil" for mapping onset_s -> model steps

rng = np.random.default_rng(0)

# Distance matrix + stimulation kernel
RHO_MM = 10.0
DIST_PATH = os.path.join(BASE, "TMS_fMRI", "atlases", "distance_matrix_450x450_Tian50_Schaefer400.npy")
D = np.load(DIST_PATH)
assert D.shape == (N, N)

# Gaussian weights
W = np.exp(-(D ** 2) / (2.0 * (RHO_MM ** 2))).astype(np.float32)
W /= (W[np.arange(N), np.arange(N)][:, None] + 1e-8)

print(f"Config: TR={TR_MODEL}s, S={S}, N={N}")
print(f"Distance matrix: {D.min():.1f}-{D.max():.1f} mm")

In [ ]:
# =========================
# 5) Helpers
# =========================
def get_onset_column(df: pd.DataFrame):
    if df is None or len(df) == 0:
        return None
    for c in ["onset", "Onset", "stim_onset", "onset_s", "onset_sec", "time", "t", "seconds"]:
        if c in df.columns:
            return c
    for c in df.columns:
        if pd.api.types.is_numeric_dtype(df[c]):
            return c
    return None

def map_onsets_to_steps(onsets_s, tr_model=TR_MODEL, mode=MAP_MODE):
    onsets_s = np.asarray(onsets_s, dtype=float)
    x = onsets_s / float(tr_model)
    if mode == "round":
        steps = np.rint(x).astype(int)
    elif mode == "floor":
        steps = np.floor(x).astype(int)
    elif mode == "ceil":
        steps = np.ceil(x).astype(int)
    else:
        raise ValueError("mode must be round|floor|ceil")
    steps = steps[steps >= 0]
    return np.unique(steps)

def safe_target_idx(target_vec):
    if target_vec is None:
        return None
    v = np.asarray(target_vec).astype(int).ravel()
    if v.size == 0 or v.sum() != 1:
        return None
    return int(np.argmax(v))

@torch.no_grad()
def predict_next(model, window_SxN: np.ndarray):
    x_np = window_SxN.reshape(-1).astype(np.float32)
    noise = NOISE_SIGMA * rng.normal(0.0, 1.0, size=x_np.shape).astype(np.float32)
    x_np = x_np + noise
    x = torch.tensor(x_np[None, :], dtype=torch.float32, device=device)
    y = model(x)
    return y.detach().cpu().numpy().squeeze(0)

def simulate_run(model, init_window_SxN, n_steps,
                 stim_steps=None, target_idx=None,
                 stim_amp=STIM_AMP,
                 burn_in=BURN_IN, W=None):
    init_window_SxN = np.asarray(init_window_SxN, dtype=np.float32)
    assert init_window_SxN.shape == (S, N)
    stim_steps = set(int(s) for s in (stim_steps or []))
    do_stim = (target_idx is not None) and (len(stim_steps) > 0)
    w = init_window_SxN.copy()

    # Burn-in
    for _ in range(burn_in):
        y = predict_next(model, w)
        w = np.vstack([w[1:], y[None, :]])

    # Session
    out = np.zeros((n_steps, N), dtype=np.float32)
    for t in range(n_steps):
        w_in = w.copy()
        if do_stim and (t in stim_steps):
            if W is None:
                w_in[-1, target_idx] += stim_amp
            else:
                w_in[-1, :] += stim_amp * W[target_idx, :]
        y = predict_next(model, w_in)
        out[t] = y
        w = np.vstack([w[1:], y[None, :]])

    meta_sim = {
        "tr_model_s": float(TR_MODEL),
        "burn_in_steps": int(burn_in),
        "noise_input_sigma": 0.3,
        "stim_amp": float(stim_amp),
        "stim_steps_modelTR": sorted(list(stim_steps)) if do_stim else [],
        "stim_mapping_mode": MAP_MODE,
    }
    return out, meta_sim

print("✅ Helpers defined")

In [ ]:
# =========================
# 6) Load per-group models
# =========================
models_cache = {}

for target_id in sorted(multi_subject_groups.keys()):
    model_path = os.path.join(MODELS_DIR, f"target_{target_id:03d}_MLP_tms_fmri.pt")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    try:
        state = torch.load(model_path, map_location=device, weights_only=True)
        model = build_model("MLP", ROI_num=N, using_steps=S).to(device)
        if isinstance(state, dict) and "state_dict" in state:
            model.load_state_dict(state["state_dict"])
        elif isinstance(state, dict):
            model.load_state_dict(state)
        else:
            raise RuntimeError("Unexpected format")
        print(f"Loaded weights with weights_only=True for target {target_id}")
    except Exception as e:
        state = torch.load(model_path, map_location=device, weights_only=False)
        model = build_model("MLP", ROI_num=N, using_steps=S).to(device)
        if isinstance(state, dict) and "state_dict" in state:
            model.load_state_dict(state["state_dict"])
        elif isinstance(state, dict):
            model.load_state_dict(state)
        else:
            model = state.to(device)
        print(f"Loaded weights with weights_only=False for target {target_id}")
    
    model.eval()
    models_cache[target_id] = model

print(f"\n✅ Loaded {len(models_cache)} per-group models")

In [ ]:
# =========================
# 7) Build dataset_simulated with per-group models
# =========================
dataset_sim = {}
n_sim_rest = 0
n_sim_stim = 0

for sub_id, sub_data in dataset_emp.items():
    dataset_sim[sub_id] = {"task-rest": {}, "task-stim": {}}
    
    # Extract subject index
    sub_name = sub_id.replace("sub-", "")
    subject_idx = subjects_list.index(sub_name) if sub_name in subjects_list else None
    
    if subject_idx is None or subject_idx not in subject_to_target:
        print(f"⚠️ {sub_id}: not in multi-subject groups, skipping")
        continue
    
    target_id = subject_to_target[subject_idx]
    model = models_cache[target_id]
    
    print(f"\n{sub_id}: target={target_id}, model loaded")
    
    # Get all stim runs for this subject
    stim_runs = sub_data.get("task-stim", {})
    n_stim_runs = len(stim_runs)
    
    # Generate REST sessions (one per stim session)
    for sim_rest_idx in range(n_stim_runs):
        # Use first empirical rest as template for duration
        emp_rest_template = None
        if "task-rest" in sub_data and len(sub_data["task-rest"]) > 0:
            emp_rest_template = list(sub_data["task-rest"].values())[0]
        
        if emp_rest_template is None:
            print(f"  ⚠️  No empirical rest template, skipping rest simulations")
            break
        
        ts_emp = emp_rest_template.get("time series", None)
        md_emp = emp_rest_template.get("metadata", {}) or {}
        
        if ts_emp is None or not isinstance(ts_emp, np.ndarray) or ts_emp.shape[1] != N:
            continue
        
        tr_emp = float(md_emp.get("tr_s", 2.0))
        dur_s = ts_emp.shape[0] * tr_emp
        n_steps = int(math.ceil(dur_s / TR_MODEL))
        
        init_window = ts_emp[:S].copy()
        sim_ts, meta_sim = simulate_run(model, init_window, n_steps)
        
        md_out = dict(md_emp)
        md_out.update({
            "simulated": True,
            "source_empirical_task": "task-rest",
            "target_id": int(target_id),
            "duration_emp_s": float(dur_s),
            "n_steps_model": int(n_steps),
            **meta_sim
        })
        
        dataset_sim[sub_id]["task-rest"][int(sim_rest_idx)] = {
            "time series": sim_ts,
            "metadata": md_out
        }
        n_sim_rest += 1
    
    # Generate STIM sessions
    for stim_run_idx, stim_run in stim_runs.items():
        ts_emp = stim_run.get("time series", None)
        md_emp = stim_run.get("metadata", {}) or {}
        target_vec = stim_run.get("target", None)
        events_df = stim_run.get("stim time", None)
        
        if ts_emp is None or not isinstance(ts_emp, np.ndarray) or ts_emp.shape[1] != N:
            continue
        
        target_idx_emp = safe_target_idx(target_vec)
        if target_idx_emp is None:
            continue
        
        onset_col = get_onset_column(events_df) if isinstance(events_df, pd.DataFrame) else None
        if onset_col is None:
            continue
        
        onsets_s = events_df[onset_col].astype(float).values
        stim_steps = list(map_onsets_to_steps(onsets_s, tr_model=TR_MODEL, mode=MAP_MODE))
        
        tr_emp = float(md_emp.get("tr_s", 2.4))
        dur_s = ts_emp.shape[0] * tr_emp
        n_steps = int(math.ceil(dur_s / TR_MODEL))
        
        init_window = ts_emp[:S].copy()
        sim_ts, meta_sim = simulate_run(model, init_window, n_steps,
                                        stim_steps=stim_steps, target_idx=target_idx_emp, W=W)
        
        md_out = dict(md_emp)
        md_out.update({
            "simulated": True,
            "source_empirical_task": "task-stim",
            "target_id": int(target_id),
            "duration_emp_s": float(dur_s),
            "n_steps_model": int(n_steps),
            "target_idx": int(target_idx_emp),
            **meta_sim
        })
        
        dataset_sim[sub_id]["task-stim"][int(stim_run_idx)] = {
            "time series": sim_ts,
            "metadata": md_out,
            "target": target_vec,
            "stim time": events_df,
        }
        n_sim_stim += 1

print(f"\n✅ Simulated {n_sim_rest} rest runs, {n_sim_stim} stim runs")

In [ ]:
# =========================
# 8) Save to Drive
# =========================
with open(OUT_PKL, "wb") as f:
    pickle.dump(dataset_sim, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved dataset_simulated to: {OUT_PKL}")

# Quick peek
some_sub = next(iter(dataset_sim.keys()), None)
if some_sub:
    print(f"\nExample subject: {some_sub}")
    print(f"task-rest runs: {list(dataset_sim[some_sub]['task-rest'].keys())[:5]}")
    print(f"task-stim runs: {list(dataset_sim[some_sub]['task-stim'].keys())[:5]}")

# Subject-Specific ΔFC Validation Test

In [ ]:
# =========================
# 9) Load data for validation
# =========================
print("Loading empirical dataset...")
with open(DATASET_EMP_PKL, "rb") as f:
    dataset_emp = pickle.load(f)

print("Loading simulated dataset...")
with open(OUT_PKL, "rb") as f:
    dataset_sim = pickle.load(f)

print(f"Loaded {len(dataset_emp)} empirical subjects")
print(f"Loaded {len(dataset_sim)} simulated subjects")

In [ ]:
# =========================
# 10) Define validation functions
# =========================
def fc_from_timeseries(ts, cortical_only=False):
    """Compute Pearson FC matrix from time series."""
    if cortical_only:
        ts = ts[:, 50:]  # Skip Tian 50, keep Schaefer 400
    return np.corrcoef(ts, rowvar=False).astype(np.float32)

def upper_tri_vec(mat, k=1):
    """Extract upper triangle as 1D vector."""
    iu = np.triu_indices(mat.shape[0], k=k)
    return mat[iu]

def pearson_corr(x, y):
    """Compute Pearson correlation."""
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    x = x - x.mean()
    y = y - y.mean()
    denom = np.sqrt(np.sum(x*x) * np.sum(y*y))
    return np.nan if denom == 0 else float(np.sum(x*y) / denom)

print("✅ Validation functions defined")

In [ ]:
# =========================
# 11) Compute per-subject ΔFC and correlations
# =========================
results = {
    'subject_deltafc_corr': {},  # sub_id -> correlation
    'subject_deltafc_vec_emp': {},  # sub_id -> vector
    'subject_deltafc_vec_sim': {},  # sub_id -> vector
}

for sub_id in sorted(dataset_emp.keys()):
    # Check if we have both empirical and simulated data
    if sub_id not in dataset_sim:
        print(f"⚠️  {sub_id}: no simulated data, skipping")
        continue
    
    sub_emp = dataset_emp[sub_id]
    sub_sim = dataset_sim[sub_id]
    
    # Get rest and stim sessions
    rest_runs_emp = sub_emp.get("task-rest", {})
    stim_runs_emp = sub_emp.get("task-stim", {})
    rest_runs_sim = sub_sim.get("task-rest", {})
    stim_runs_sim = sub_sim.get("task-stim", {})
    
    if len(rest_runs_emp) == 0 or len(stim_runs_emp) == 0:
        print(f"⚠️  {sub_id}: missing rest or stim sessions, skipping")
        continue
    
    if len(rest_runs_sim) == 0 or len(stim_runs_sim) == 0:
        print(f"⚠️  {sub_id}: missing simulated sessions, skipping")
        continue
    
    # Average rest FC (empirical and simulated)
    fc_rest_emp_list = []
    for run in rest_runs_emp.values():
        ts = run.get("time series", None)
        if isinstance(ts, np.ndarray) and ts.shape[1] >= 450:
            fc_rest_emp_list.append(fc_from_timeseries(ts, cortical_only=True))
    
    fc_rest_sim_list = []
    for run in rest_runs_sim.values():
        ts = run.get("time series", None)
        if isinstance(ts, np.ndarray) and ts.shape[1] >= 450:
            fc_rest_sim_list.append(fc_from_timeseries(ts, cortical_only=True))
    
    if len(fc_rest_emp_list) == 0 or len(fc_rest_sim_list) == 0:
        print(f"⚠️  {sub_id}: no valid rest sessions, skipping")
        continue
    
    FC_rest_emp = np.mean(np.stack(fc_rest_emp_list), axis=0)
    FC_rest_sim = np.mean(np.stack(fc_rest_sim_list), axis=0)
    
    # Average stim FC
    fc_stim_emp_list = []
    for run in stim_runs_emp.values():
        ts = run.get("time series", None)
        if isinstance(ts, np.ndarray) and ts.shape[1] >= 450:
            fc_stim_emp_list.append(fc_from_timeseries(ts, cortical_only=True))
    
    fc_stim_sim_list = []
    for run in stim_runs_sim.values():
        ts = run.get("time series", None)
        if isinstance(ts, np.ndarray) and ts.shape[1] >= 450:
            fc_stim_sim_list.append(fc_from_timeseries(ts, cortical_only=True))
    
    if len(fc_stim_emp_list) == 0 or len(fc_stim_sim_list) == 0:
        print(f"⚠️  {sub_id}: no valid stim sessions, skipping")
        continue
    
    FC_stim_emp = np.mean(np.stack(fc_stim_emp_list), axis=0)
    FC_stim_sim = np.mean(np.stack(fc_stim_sim_list), axis=0)
    
    # Compute ΔFC
    deltaFC_emp = FC_stim_emp - FC_rest_emp
    deltaFC_sim = FC_stim_sim - FC_rest_sim
    
    # Extract upper triangle
    vec_emp = upper_tri_vec(deltaFC_emp, k=1)
    vec_sim = upper_tri_vec(deltaFC_sim, k=1)
    
    # Correlate
    r = pearson_corr(vec_emp, vec_sim)
    
    results['subject_deltafc_corr'][sub_id] = r
    results['subject_deltafc_vec_emp'][sub_id] = vec_emp
    results['subject_deltafc_vec_sim'][sub_id] = vec_sim
    
    print(f"{sub_id}: r_ΔFC = {r:.4f}")

print(f"\n✅ Computed per-subject ΔFC correlations for {len(results['subject_deltafc_corr'])} subjects")

In [ ]:
# =========================
# 12) Summary statistics
# =========================
corrs = np.array(list(results['subject_deltafc_corr'].values()))
corrs_valid = corrs[np.isfinite(corrs)]

print("\n" + "="*70)
print("PER-SUBJECT ΔFC CORRELATION SUMMARY")
print("="*70)
print(f"N subjects: {len(corrs_valid)}")
print(f"Mean r: {corrs_valid.mean():.4f}")
print(f"Std r: {corrs_valid.std():.4f}")
print(f"Median r: {np.median(corrs_valid):.4f}")
print(f"Min r: {corrs_valid.min():.4f}")
print(f"Max r: {corrs_valid.max():.4f}")
print(f"\nAll correlations:")
for sub_id, r in sorted(results['subject_deltafc_corr'].items()):
    print(f"  {sub_id}: {r:.4f}")

In [ ]:
# =========================
# 13) Visualization: Histogram of correlations
# =========================
fig, ax = plt.subplots(figsize=(8, 5))

ax.hist(corrs_valid, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(corrs_valid.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean = {corrs_valid.mean():.3f}')
ax.axvline(np.median(corrs_valid), color='green', linestyle='--', linewidth=2, label=f'Median = {np.median(corrs_valid):.3f}')

ax.set_xlabel('Per-Subject ΔFC Correlation (r)', fontsize=12)
ax.set_ylabel('Number of Subjects', fontsize=12)
ax.set_title('Subject-Specific ΔFC: Empirical vs. Simulated (Per-Group Models)', fontsize=13)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# =========================
# 14) Visualization: Scatter plot (example subject)
# =========================
if len(results['subject_deltafc_vec_emp']) > 0:
    example_sub = list(results['subject_deltafc_vec_emp'].keys())[0]
    vec_emp = results['subject_deltafc_vec_emp'][example_sub]
    vec_sim = results['subject_deltafc_vec_sim'][example_sub]
    r = results['subject_deltafc_corr'][example_sub]
    
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(vec_emp, vec_sim, s=10, alpha=0.5, color='steelblue')
    
    # Add regression line
    m, b = np.polyfit(vec_emp, vec_sim, 1)
    x_line = np.array([vec_emp.min(), vec_emp.max()])
    ax.plot(x_line, m*x_line + b, 'r--', linewidth=2, label=f'r = {r:.3f}')
    
    ax.set_xlabel('Empirical ΔFC', fontsize=11)
    ax.set_ylabel('Simulated ΔFC', fontsize=11)
    ax.set_title(f'Example: {example_sub} (Per-Group Model)', fontsize=12)
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# =========================
# 15) Save validation results
# =========================
results_summary = {
    'n_subjects': len(corrs_valid),
    'mean_r_deltafc': float(corrs_valid.mean()),
    'std_r_deltafc': float(corrs_valid.std()),
    'median_r_deltafc': float(np.median(corrs_valid)),
    'min_r_deltafc': float(corrs_valid.min()),
    'max_r_deltafc': float(corrs_valid.max()),
    'per_subject_correlations': {k: float(v) for k, v in results['subject_deltafc_corr'].items()},
}

results_path = os.path.join(OUT_DIR, "deltafc_validation_results.json")
with open(results_path, "w") as f:
    json.dump(results_summary, f, indent=2)

print(f"💾 Saved validation results to: {results_path}")
print(f"\n✅ All done!")